In [1]:
from steganogan import SteganoGAN
from steganogan.critics import BasicCritic
from steganogan.decoders import BasicDecoder, DenseDecoder
from steganogan.encoders import BasicEncoder, DenseEncoder
from steganogan.loader import DataLoader

In [2]:
# Load the data
train = DataLoader('data/div2k/train/')
validation = DataLoader('data/div2k/val/')

In [3]:
# Create the SteganoGAN instance
steganogan = SteganoGAN(1, BasicEncoder, BasicDecoder, BasicCritic, hidden_size=32, cuda=True, verbose=True)

Using CUDA device


In [4]:
# Fit on the given data
steganogan.fit(train, validation, epochs=10)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/10


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 2/10


  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 9/10


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 10/10


100%|██████████| 25/25 [00:05<00:00,  5.12it/s]


In [5]:
# Save the fitted model
steganogan.save('models/basic.steg')

In [6]:
# Load the model
steganogan = SteganoGAN.load('models/basic.steg')

Using CUDA device


In [7]:
# Encode a message in input.png
steganogan.encode('input.png', 'output.png', 'This is a super secret message!')

Encoding completed.


In [8]:
# Decode the message from output.png
steganogan.decode('output.png')

'This is a super secret message!'